# JEM092 - Asset Pricing HW1 - Pavlína Křenková & Tomáš Svoboda

In [2]:
# libraries
shhh = suppressPackageStartupMessages
shhh(library(quantmod))
shhh(library(sandwich))
shhh(library(xts))
shhh(library(lubridate))

# 1. Introduction

This analysis aims to find an effect of three sorting variables (size, beta, book-to-market) on company's future returns. We begin with downloading all the necesarry data and adjust them to the appropriate formats. All 250 stock are then divided into 5 portfolios based on the all sorting variables and the average returns are calculated employing both equally and value-weighted methods. The main goal of this analysis is to confirm standard theoretical assumptions about the effect direction of the sorting variables on stock returns and to show is any of them can be used for predictions. 

# 2. Data

We begin with loading all the necessary data for the analysis. First of all, data for monthly FF3 factors, daily FF5 factors, Momentum, and Sort-term reversal are downloaded from the respective website (https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html) and set into the notebook. Reading of the files is done individually for each dataset due to their unique composition. Downloading and setting of the dataset for liquidity from the website (https://faculty.chicagobooth.edu/lubos-pastor/data) is then proceed analogically. Finally, we loaded also the datasets for price-volumes, book values and market capitalizations prepared during HA1.   

### a) Upload the data

Download the dataset from the website

In [3]:
rm(list = ls()) ; par(mfrow = c(1,1))

# Monthly FF3 factors
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_TXT.zip",
    destfile = "F-F_Research_Data_Factors.zip"
)
# Daily FF5 factors
download.file(
    'http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_5_Factors_2x3_daily_TXT.zip',
    destfile = 'F-F_Research_Data_5_Factors_2x3_daily_TXT.zip'
)
# Momentum
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Momentum_Factor_TXT.zip",
    destfile = "F-F_Momentum_Factor.zip"
)
# ST Reversal
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_ST_Reversal_Factor_TXT.zip",
    destfile = "F-F_ST_Reversal_Factor.zip"
)

unzip("F-F_Research_Data_Factors.zip")
unzip('F-F_Research_Data_5_Factors_2x3_daily_TXT.zip')
unzip("F-F_Momentum_Factor.zip")
unzip("F-F_ST_Reversal_Factor.zip")


Monthly FF3 factors

In [4]:
# Monthly 3 factors
FF3factors <- read.delim(
    'F-F_Research_Data_Factors.txt',
    col.names = c('t', 'mkt.rf', 'smb', 'hml', 'rf'), 
    sep = '',
    nrows = 1170,
    header = FALSE,
    skip = 4,
    stringsAsFactors = FALSE
)
# Set date as index
FF3factors$t <- as.Date(as.yearmon(as.character(FF3factors$t), format = "%Y%m"), frac = 1)
FF3factors_xts <- xts(FF3factors[, -1], order.by = FF3factors$t)

Daily FF5 factors

In [5]:
# Daily 5 factors data -> used for calculating daily excess returns 
FF5_factors <- read.delim(
    'F-F_Research_Data_5_Factors_2x3_daily.txt',
    col.names = c('t', 'mkt_rf', 'smb', 'hml', 'rmw', 'cma', 'rf'),
    sep = '',
    nrows = 25438,
    header = FALSE,
    skip = 5,
    stringsAsFactors = FALSE
)
FF5_factors$t <- as.Date(as.character(FF5_factors$t), format='%Y%m%d')
FF5_factors_xts <- as.xts(FF5_factors[, 2:7], order.by = FF5_factors$t)

Momentum

In [6]:
# Momentum
FFmom <- read.delim(
    'F-F_Momentum_Factor.txt',
    col.names = c('t', 'mom'), 
    sep = '',
    nrows = 1164,
    header = TRUE,
    skip = 13,
    stringsAsFactors = FALSE
)
# Set date as index
FFmom$t <- as.Date(as.yearmon(as.character(FFmom$t), format = "%Y%m"), frac = 1)
FFmom_xts <- xts(FFmom[, -1], order.by = FFmom$t)
colnames(FFmom_xts) <- "mom"

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"header and 'col.names' are of different lengths"


Short-term reversal

In [7]:
# Load Short-term reversal
FFstrev <- read.delim(
    'F-F_ST_Reversal_Factor.txt',
    col.names = c('t', 'ST_Rev'), 
    sep = '',
    nrows = 1175,
    header = TRUE,
    skip = 12,
    stringsAsFactors = FALSE
)
# Set date as index
FFstrev$t <- as.Date(as.yearmon(as.character(FFstrev$t), format = "%Y%m"), frac = 1)
FFstrev_xts <- xts(FFstrev[, -1], order.by = FFstrev$t)
colnames(FFstrev_xts) <- "strev"

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"header and 'col.names' are of different lengths"


Liquidity factor

In [8]:
# Load Short-term reversal
liq_factor <- read.delim(
    'liquidity_factor.txt',
    col.names = c('Month', 'Agg Liq.', 'Innov Liq (eq8)', 'Traded Liq (LIQ_V)'), 
    sep = '',
    header = TRUE,
    skip = 10,
    stringsAsFactors = FALSE
)
# Set date as index
liq_factor$Month <- as.Date(as.yearmon(as.character(liq_factor$Month), format = "%Y%m"), frac = 1)
liq_factor_xts <- xts(liq_factor[, -1], order.by = liq_factor$Month)

tail(liq_factor_xts)

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"header and 'col.names' are of different lengths"


              Agg.Liq. Innov.Liq..eq8. Traded.Liq..LIQ_V.
2023-07-31  0.03158360      0.05313238        -0.03521294
2023-08-31 -0.02219315     -0.01384346        -0.04206628
2023-09-30 -0.06412711     -0.03281943        -0.04053745
2023-10-31  0.16340711      0.17621454         0.00078403
2023-11-30  0.01836070     -0.10736516         0.05513213
2023-12-31 -0.09925067     -0.12738569         0.03728561

Price-volume, book value, market capitalization from HA1

In [9]:
price_volume_xts <- readRDS("price_volume_xts.rds")
book_value_xts <- readRDS("book_value_xts.rds")
market_cap_xts <- readRDS("market_cap_xts.rds") # Lists of xts objects

### b) Data adjustments

Unifying time intervals for all datasets to 1.1.2010 - 31.12.2023

In [10]:
start_date <- as.Date("2010-01-01")
end_date <- as.Date("2023-12-31")

adjust_sample_period <- function(xts_series) {
    lapply(xts_series, function(stock){
        stock[paste(start_date, "/", end_date, sep = "")]
    })
}

price_volume <- adjust_sample_period(price_volume_xts)
book_value <- adjust_sample_period(book_value_xts)
market_cap <- adjust_sample_period(market_cap_xts)

three_factors <- adjust_sample_period(FF3factors_xts)
five_factors <- adjust_sample_period(FF5_factors_xts)
momentum_factor <- adjust_sample_period(FFmom_xts)
strev_factor <- adjust_sample_period(FFstrev_xts)

liquidity_factor <- adjust_sample_period(liq_factor_xts)

Converting all variables from lists to xts data frames with the same format. Moreover, book-to-market ratio is calculated according to the formula $$book.to.market_{i,t} = \frac{book.value.per.share_{i,t}}{stock.price_{i,t}}$$ where i and t means company and time interval, respectively.

In [11]:
# Extract the values
price_list <- lapply(price_volume, function(x) x[,1])
volume_list <- lapply(price_volume, function(x) x[,2])
book_to_market_list <- lapply(book_value, function(x) x[,2]/x[,1]) # Book to market ratio calculation

# Convert lists to dataframes
prices <- do.call('cbind', price_list)
volumes <- do.call('cbind', volume_list)
book_to_market <- do.call('cbind', book_to_market_list)
nominals <- do.call('cbind', market_cap)

three_fact <- do.call('cbind', three_factors)
five_fact <- do.call('cbind', five_factors)
momentum <- momentum_factor$mom
strev <- strev_factor$strev
liquidity <- liquidity_factor$Traded.Liq..LIQ_V. # Traded liquidity factor (LIQ_V)

# Set column names
colnames(prices) <- paste(names(price_list), 'prices', sep='.')
colnames(volumes) <- paste(names(volume_list), 'volumes', sep='.')
colnames(book_to_market) <- paste(names(book_to_market_list), 'book_to_market', sep='.')
colnames(nominals) <- paste(names(market_cap), 'nominals', sep='.')

colnames(liquidity) <- "liquidity"

### c) Monthly returns, volatilities and market capitalizations  

In this section, daily and monthly returns were calculated using the stock prices. We also calculated the excess returns that will be used further in the analysis. The excess returns both monthly and daily are calculated as $$excess.returns_{i,t} = returns_{i,t} - risk.free.rate_{t}$$ Furthermore, we also calculated monthly volatilities (out of daily data), monthly nominals/market capitalizations (out of monthly data), and monthly book-to-market ratios (out of monthly data). 

In [12]:
# Monthly & daily returns
daily_returns <- lapply(prices, function(x) suppressWarnings(dailyReturn(x)))
monthly_returns <- lapply(prices, function(x) suppressWarnings(monthlyReturn(x)))
daily_returns <- do.call('cbind', daily_returns)
monthly_returns <- do.call('cbind', monthly_returns)
colnames(daily_returns) <- paste(names(price_list), 'dreturns', sep='.')
colnames(monthly_returns) <- paste(names(price_list), 'mreturns', sep='.')
index(monthly_returns) <- as.Date(as.yearmon(index(monthly_returns), format = "Q%q/%y"), frac = 1)

# Excess returns
for (i in 1:ncol(monthly_returns)) {
    daily_returns[, i] <- daily_returns[, i] - five_fact[, "rf"]
    monthly_returns[, i] <- monthly_returns[, i] - three_fact[, "rf"]
}


# Monthly volatilities
monthly_volatilities <- as.xts(aggregate(daily_returns, as.yearmon(time(daily_returns)), sd)) # Extract time index from daily_returns, convert it to 'yearmon' format, then aggregate/group daily returns with month times, then calculates sd within each month, then convert to xts object
index(monthly_volatilities) <- as.Date(as.yearmon(index(monthly_volatilities), format = "Q%q/%y"), frac = 1)
colnames(monthly_volatilities) <- paste(names(price_list), 'mvolatilities', sep='.')

# Monthly nominals
monthly_nominals <- as.xts(aggregate(nominals, as.yearmon(time(nominals)), sum))
index(monthly_nominals) <- as.Date(as.yearmon(index(monthly_nominals), format = "Q%q/%y"), frac = 1)
colnames(monthly_nominals) <- paste(names(price_list), 'mnominals', sep='.')

# Monthly book_to_market (convert quarterly to monthly)
monthly_book_to_market <- na.locf(book_to_market, na.rm=FALSE) # Fill NAs with last observation (best available assumption)
monthly_book_to_market <- na.locf(monthly_book_to_market, fromLast=TRUE) # Fill also the first 2 NAs for values starting in March to prevent NAs
index(monthly_book_to_market) <- as.Date(as.yearmon(index(monthly_book_to_market), format = "Q%q/%y"), frac = 1)
dup_indices <- which(index(monthly_book_to_market) == as.Date("2018-09-30")) # Index "2018-09-30" appears twice => remove the second one
monthly_book_to_market <- monthly_book_to_market[-dup_indices[2], ] # Remove the second index

### d) Sizes

We proceed with computing stock sizes using the formula: $$size_{i,t} = ln(market.cap_{i,t})$$
Sizes (monthly_sizes) are then used in the remaining analysis instead of simple market capitalizations (monthly_nominals)

In [13]:
monthly_sizes <- log(monthly_nominals)
colnames(monthly_sizes) <- paste(names(price_list), 'msizes', sep='.')

# Replace -Inf and Inf in `monthly_sizes` with NAs.
monthly_sizes <- as.xts(apply(monthly_sizes, 2, function(x) ifelse(is.finite(x), x, NA)))

### e) Beta

We proceed with estimating betas using the following regression equation:

$r_{i,t} = \alpha_i + \beta_i MKT_t + \epsilon_{i,t}$

- $i$ denotes stocks
- $t$ denotes time
- $r$ is the excess stock return
- $MKT$ is the excess market return
- $\alpha$ is the intercept
- $\beta$ is the slope coefficient
- $\epsilon$ is the error term

In [14]:
n_months_betas = 12 # Last 12 months

# Initialize an xts object for storing betas.
monthly_betas <- as.xts(
    matrix(
        nrow = nrow(monthly_returns),
        ncol = ncol(monthly_returns)
    ),
    order.by = index(monthly_returns)
)
names(monthly_betas) <- paste(names(price_list), 'mbetas', sep='.')

# Iterate over rows and compute betas.
for (i in 1:nrow(monthly_betas)) {
    end_day <- index(monthly_betas[i]) # Date of respective i-th row
    start_day <- end_day %m-% months(n_months_betas) # Start date 12 months before end date
    dates <- paste0(start_day, '/', end_day) # => 12 months windows

    returns <- daily_returns[dates] # Filtering the data in a specific time window
    market_returns <- five_fact[dates]

    # Estimate betas for each stock.
    for (j in 1:ncol(monthly_betas)) {
        stock_returns <- returns[, j]
        # Merge excess returns and market excess returns.
        model_data <- cbind(stock_returns, market_returns[, "mkt_rf"])
        model_data <- na.omit(model_data)

        # Don't estimate betas if the number of observations is lower than n_months_betas * 15.
        if (nrow(model_data) < n_months_betas * 15) {
            next
        }

        model_betas <- lm(model_data[, 1] ~ model_data[, 2])
        # Save the given beta to `monthly_betas`.
        monthly_betas[i, j] <- model_betas$coefficients[[2]]
    }
}

### f) Overview 

Finally, all factors are merged into one dataframe the resulting datasets from the sections above are presented

In [15]:
factors <- cbind(three_fact, momentum, strev, liquidity)

In [16]:
monthly_returns[1:5, 1:3]
monthly_volatilities[1:5, 1:3]
monthly_betas[21:25, 1:3]
monthly_nominals[1:5, 1:3]
monthly_sizes[1:5, 1:3]
monthly_book_to_market[1:5, 1:3]

factors[1:5]

           UDR.mreturns CPB.mreturns MTB.mreturns
2010-01-31 -0.028893357 -0.022438759   0.10058214
2010-02-28  0.079691093  0.006644808   0.05950287
2010-03-31  0.039999893  0.058855367   0.01518418
2010-04-30  0.152766732  0.004427361   0.09040268
2010-05-31 -0.009015349 -0.011394121  -0.09486931

           UDR.mvolatilities CPB.mvolatilities MTB.mvolatilities
2010-01-31        0.01656583       0.009732334        0.02461843
2010-02-28        0.01975318       0.011155031        0.01989432
2010-03-31        0.01102678       0.007085745        0.01227059
2010-04-30        0.02366237       0.005880367        0.02009457
2010-05-31        0.03144443       0.012626875        0.02886656

           UDR.mbetas  CPB.mbetas MTB.mbetas
2011-09-30 0.01108075 0.004154014 0.01003054
2011-10-31 0.01166690 0.003854462 0.01043471
2011-11-30 0.01136438 0.003913114 0.01079158
2011-12-31 0.01130378 0.003958241 0.01085015
2012-01-31 0.01123826 0.003947312 0.01094114

           UDR.mnominals CPB.mnominals MTB.mnominals
2010-01-31         46.58        214.98        164.96
2010-02-28         47.10        214.49        166.65
2010-03-31         62.52        270.62        217.82
2010-04-30         66.23        253.27        211.25
2010-05-31         65.74        241.72        200.58

           UDR.msizes CPB.msizes MTB.msizes
2010-01-31   3.841171   5.370545   5.105703
2010-02-28   3.852273   5.368263   5.115896
2010-03-31   4.135487   5.600716   5.383669
2010-04-30   4.193134   5.534456   5.353042
2010-05-31   4.185708   5.487780   5.301213

           UDR.book_to_market CPB.book_to_market MTB.book_to_market
2010-01-31           0.813197          0.1397601           1.248315
2010-02-28           0.813197          0.1397601           1.248315
2010-03-31           0.813197          0.1397601           1.248315
2010-04-30           0.813197          0.1369400           1.248315
2010-05-31           0.813197          0.1369400           1.248315

           mkt.rf  smb   hml   rf   mom strev   liquidity
2010-01-31  -3.36 0.40  0.43 0.00 -5.40  3.64 -0.01105698
2010-02-28   3.40 1.19  3.23 0.00  3.74  1.63 -0.00755895
2010-03-31   6.31 1.48  2.21 0.01  3.76  0.80 -0.06257515
2010-04-30   2.00 4.87  2.89 0.01  3.16 -1.86  0.01020015
2010-05-31  -7.89 0.09 -2.44 0.01 -0.25  0.52 -0.02806075

# 3. Methodology

In this section, several portfolios sorted by three variables (beta, size, book-to-market ratio) were created while averaging the final returns using the equal-weighted and value-weighted mathod. 

These portfolios were than used for calculating the next-month average returns using a simple average, CAPM model, and the FF model. FInally, the Fama-MacBeth regression is performed to analyze an effect of size to the future returns. 

## a) Univariate portfolio sort

Function 'portfolio_returns' is expected to align all the explored companies based on their market capitalization using the monthly rebalancing. These stocks are then divided into 5 portfolios plus one additional difference portfolio constructed as a difference between the first and last portfolio. In this way, companies contained in these portfolios differ in their size on average. Returns for every month and every portfolio are then averaged using either equal weights or weights based on companies market capitalization. Weighting method can be set as an argument either to "VW" for value-weighted method or "EW" for the equal-weighted method.

In [17]:
portfolio_returns <- function(monthly_sort_variable, type, num_portfolios=5, monthly_ret=monthly_returns, monthly_nom=monthly_nominals) {
    n_portfolios <- num_portfolios # number of portfolios
    diff_portfolio <- paste(n_portfolios, '- 1') # "5 - 1" portfolio

    # Initialize an xts object for storing portfolio returns.
    portfolio_ret <- as.xts( 
        matrix(
            nrow = nrow(monthly_sort_variable) - 1,
            ncol = n_portfolios + 1 # Last one for Diff portfolio
        ),
        order.by = index(monthly_sort_variable[2:nrow(monthly_sort_variable)]) # Order indexes
    )
    names(portfolio_ret) <- c(1:n_portfolios, diff_portfolio) # Add column names

    # Iterate over rows, find breakpoints and compute month returns within the given nominal bounds.
    for (i in 1:nrow(portfolio_ret)) { # For every month
        current_month <- index(portfolio_ret[i])
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

        if (!next_month %in% index(monthly_ret)) {
            next
        }
        if (type == "VW") {
            if (!next_month %in% index(monthly_nom)) {
                next
            }        
        }

        # Avoid look-ahead bias by sorting the returns after the month used to compute breakpoints ends.
        month_returns <- monthly_ret[next_month] # Returns in next month
        month_market_caps <- monthly_nom[next_month] # Market caps in next month

        # Replace NA market caps with 0s so that such observations have no weights.
        month_market_caps[is.na(month_market_caps)] <- 0
        month_sort_variable <- monthly_sort_variable[current_month] # Sort variable for the respective month

        # Breakpoints        
        breakpoints <- quantile(month_sort_variable, 0:n_portfolios/n_portfolios, na.rm = TRUE) # Find market cap breakpoints for the respective month
        breakpoints[1] <- breakpoints[1] - 1 # Adjust min
        breakpoints[length(breakpoints)] <- breakpoints[length(breakpoints)] + 1 # Adjust max
        not_na <- !is.na(month_sort_variable) # Which stocks dont have NAs in market caps

        for (j in 1:n_portfolios) { # For every portfolio
            filter <- (breakpoints[[j]] <= month_sort_variable) & (month_sort_variable < breakpoints[[j + 1]]) & not_na # Find stocks that have market cap between the breakpoints and are not NA 
            if (type == "VW") {
                portfolio_ret[i, j] <- weighted.mean(t(month_returns[, filter]), t(month_market_caps[, filter]), na.rm = TRUE) # Calculate the mean return of the stocks in a respective portfolio
            } else if (type == "EW") {
                portfolio_ret[i, j] <- mean(month_returns[, filter], na.rm = TRUE)
            } else {
                stop("Incorrect portfolio-weighting method")
            }
        }
        portfolio_ret[i, diff_portfolio] <- portfolio_ret[i, n_portfolios] - portfolio_ret[i, 1] # Define the Diff column (= last - first portfolio)
    }

    return (portfolio_ret)
}

#### (i) Equally-weighted portfolio

In [18]:
# Mean returns for each month per equally-weighted portfolio
EW_portfolio_returns_betas <- portfolio_returns(monthly_betas, type="EW")
EW_portfolio_returns_size <- portfolio_returns(monthly_sizes, type="EW")
EW_portfolio_returns_book_to_market <- portfolio_returns(monthly_book_to_market, type="EW")

print("EW: Betas")
EW_portfolio_returns_betas[21:26,]
print("EW: Size")
EW_portfolio_returns_size[1:6,]
print("EW: Book-to-Market")
EW_portfolio_returns_book_to_market[1:6,]

[1] "EW: Betas"


                    1           2            3             4           5
2011-10-31 0.01544986 0.007763864 -0.008118379 -0.0013459011 -0.01203951
2011-11-30 0.03758664 0.014571626  0.002093008  0.0008615447 -0.02436879
2011-12-31 0.01021286 0.048883148  0.075896088  0.0913579764  0.12026834
2012-01-31 0.02142837 0.042682026  0.042585039  0.0516161762  0.05212398
2012-02-29 0.04094933 0.038632089  0.027795531  0.0430094112  0.01306961
2012-03-31 0.02016014 0.005281395  0.012759866 -0.0197271411 -0.02514970
                 5 - 1
2011-10-31 -0.02748937
2011-11-30 -0.06195544
2011-12-31  0.11005547
2012-01-31  0.03069560
2012-02-29 -0.02787971
2012-03-31 -0.04530984

[1] "EW: Size"


                     1           2           3           4            5
2010-02-28  0.07703300  0.05561677  0.05949845  0.05274118  0.050600256
2010-03-31  0.04185189  0.03076809  0.04019638  0.02875165  0.008814152
2010-04-30 -0.06043563 -0.06760921 -0.06931802 -0.07911023 -0.102290039
2010-05-31 -0.05959897 -0.05328542 -0.05374921 -0.05078681 -0.063712570
2010-06-30  0.05505042  0.07075867  0.05380908  0.05114000  0.064362055
2010-07-31 -0.06408788 -0.04626697 -0.04015706 -0.04430756 -0.064432475
                   5 - 1
2010-02-28 -0.0264327447
2010-03-31 -0.0330377419
2010-04-30 -0.0418544140
2010-05-31 -0.0041136025
2010-06-30  0.0093116305
2010-07-31 -0.0003445956

[1] "EW: Book-to-Market"


                     1           2           3           4           5
2010-02-28  0.06842814  0.04814306  0.05553151  0.06038915  0.06177105
2010-03-31  0.01762683  0.02379054  0.02701314  0.05052288  0.02776845
2010-04-30 -0.06662108 -0.07926554 -0.08017189 -0.07248483 -0.07967147
2010-05-31 -0.05215395 -0.05740655 -0.07266388 -0.05944958 -0.04094185
2010-06-30  0.06625483  0.05559293  0.05402006  0.04611421  0.06354305
2010-07-31 -0.03757711 -0.05090537 -0.07030621 -0.05379222 -0.04464189
                  5 - 1
2010-02-28 -0.006657086
2010-03-31  0.010141620
2010-04-30 -0.013050388
2010-05-31  0.011212097
2010-06-30 -0.002711776
2010-07-31 -0.007064777

#### (ii) Value-weighted portfolio

In [19]:
# Mean returns for each month per value-weighted portfolio
VW_portfolio_returns_betas <- portfolio_returns(monthly_betas, type="VW")
VW_portfolio_returns_size <- portfolio_returns(monthly_sizes, type="VW")
VW_portfolio_returns_book_to_market <- portfolio_returns(monthly_book_to_market, type="VW")

print("VW: Betas")
VW_portfolio_returns_betas[21:26,]
print("VW: Size")
VW_portfolio_returns_size[1:6,]
print("VW: Book-to-Market")
VW_portfolio_returns_book_to_market[1:6,]

[1] "VW: Betas"


                       1             2            3           4           5
2011-10-31  0.0170229935 -0.0076970381 -0.001116119 -0.01969016 -0.02303189
2011-11-30  0.0420173728  0.0185941818  0.029297515 -0.04604255 -0.04355001
2011-12-31  0.0296184106  0.0579788793  0.065799009  0.09162488  0.12781670
2012-01-31  0.0532867835  0.0732001014  0.032562230  0.04574516  0.06171436
2012-02-29  0.0573363945  0.0323779411  0.034457292  0.03354901  0.00254337
2012-03-31 -0.0006029864  0.0008682739  0.002433666 -0.01548847 -0.03489524
                  5 - 1
2011-10-31 -0.040054883
2011-11-30 -0.085567378
2011-12-31  0.098198286
2012-01-31  0.008427574
2012-02-29 -0.054793025
2012-03-31 -0.034292255

[1] "VW: Size"


                     1           2           3           4           5
2010-02-28  0.07663648  0.05924961  0.06122816  0.05599392  0.05603798
2010-03-31  0.04494215  0.03352383  0.04271117  0.03220183  0.01339041
2010-04-30 -0.05216451 -0.06441340 -0.06767518 -0.07747537 -0.10499117
2010-05-31 -0.06453471 -0.05236156 -0.05253849 -0.05010297 -0.06119919
2010-06-30  0.06169450  0.07515980  0.05919784  0.05410926  0.05877881
2010-07-31 -0.06458714 -0.04415801 -0.03554519 -0.03982683 -0.06956371
                  5 - 1
2010-02-28 -0.020598497
2010-03-31 -0.031551736
2010-04-30 -0.052826667
2010-05-31  0.003335522
2010-06-30 -0.002915691
2010-07-31 -0.004976566

[1] "VW: Book-to-Market"


                     1            2           3           4           5
2010-02-28  0.05889617  0.044680077  0.04686059  0.05924369  0.07697548
2010-03-31  0.03717130  0.009962324  0.01267681  0.01911219  0.02288306
2010-04-30 -0.08504183 -0.107806772 -0.08113236 -0.08730215 -0.10463081
2010-05-31 -0.05962666 -0.051415897 -0.07132990 -0.05416487 -0.06190217
2010-06-30  0.06602405  0.068970251  0.03826860  0.05298948  0.05634993
2010-07-31 -0.06102793 -0.053081858 -0.04848281 -0.05254358 -0.08624413
                  5 - 1
2010-02-28  0.018079309
2010-03-31 -0.014288244
2010-04-30 -0.019588981
2010-05-31 -0.002275505
2010-06-30 -0.009674116
2010-07-31 -0.025216200

## b) Average values

Average values of each sorting variable are calculated and presented in this section using all 5 portfolios. Function 'portfolio_results' accepts previously calculated portfolio returns and reveals a table with average returns within each portfolio and respective t-statistics. Average returns are estimated by regressing mean returns of each portfolio with respect to the intercept, capm-alpha is estimated using the standard CAPM model regression, and finally, the first coefficient of the factor model gives us the last factor-alpha estimate. NeweyWest method is then emplyed for t-statistics calculations. All three average values estimates are then collected into one table for each weighted method. 

In [20]:
# Compute overall average returns within portfolio, their standard errors and other stats.
portfolio_results <- function(portfolio_returns, num_portfolios=5, fact=factors) {
    n_portfolios <- num_portfolios # number of portfolios
    diff_portfolio <- paste(n_portfolios, '- 1') # "5 - 1" portfolio
    factors_aligned <- fact[-1,] # Remove the first row of factors to meet portfolio_returns dimensions (for index correction in lm)
    
    portfolio_res <- as.data.frame(
        matrix(nrow = 6, ncol = n_portfolios + 1),
        row.names = c(
            'average return',
            'average return t-stat',
            'CAPM alpha',
            'CAPM alpha t-stat',
            'FF alpha',
            'FF alpha t-stat'
        )
    )
    names(portfolio_res) <- c(1:n_portfolios, diff_portfolio)

    for (i in 1:ncol(portfolio_res)) {
        model <- lm(na.omit(portfolio_returns[, i]) ~ 1)
        portfolio_res[1, i] <- model$coefficients[[1]]
        portfolio_res[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6)[[1]])
        is_not_na_filter = rowSums(is.na(portfolio_returns)) == 0
        index(portfolio_returns) <- index(factors_aligned) # index correction of portfolio_returns (lm returns different indexing)
        model_capm <- lm(
            portfolio_returns[is_not_na_filter, i] ~ (
                1
                + fact[index(portfolio_returns[is_not_na_filter])]$mkt.rf
            )
        )
        portfolio_res[3, i] <- model_capm$coefficients[[1]]
        portfolio_res[4, i] <- model_capm$coefficients[[1]] / sqrt(NeweyWest(model_capm, lag = 6)[[1]])

        # OLS with respect to factors
        model_factors <- lm(
            portfolio_returns[is_not_na_filter, i] ~ ( # regress on all filters
                1
                + fact[index(portfolio_returns[is_not_na_filter])]$mkt.rf
                + fact[index(portfolio_returns[is_not_na_filter])]$smb
                + fact[index(portfolio_returns[is_not_na_filter])]$hml
                + fact[index(portfolio_returns[is_not_na_filter])]$mom
                + fact[index(portfolio_returns[is_not_na_filter])]$strev
                + fact[index(portfolio_returns[is_not_na_filter])]$liquidity
            )
        )
        portfolio_res[5, i] <- model_factors$coefficients[[1]]
        portfolio_res[6, i] <- model_factors$coefficients[[1]] / sqrt(NeweyWest(model_factors, lag = 6)[[1]])
    }
    
    return (portfolio_res)
}

#### (i) Equally-weighted portfolio

In [21]:
# Overall average returns within equally-weighted portfolios, their standard errors and other stats.
EW_portfolio_results_betas <- portfolio_results(EW_portfolio_returns_betas)
EW_portfolio_results_size <- portfolio_results(EW_portfolio_returns_size)
EW_portfolio_results_book_to_market <- portfolio_results(EW_portfolio_returns_book_to_market)

#### (ii) Value-weighted portfolio

In [22]:
# Overall average returns within value-weighted portfolios, their standard errors and other stats.
VW_portfolio_results_betas <- portfolio_results(VW_portfolio_returns_betas)
VW_portfolio_results_size <- portfolio_results(VW_portfolio_returns_size)
VW_portfolio_results_book_to_market <- portfolio_results(VW_portfolio_returns_book_to_market)

## c) Fama-MacBeth regression

In the last section of our analysis, Fama-MacBeth regression model is employed to estimate the effect of the stock size on its future returns. Firstly, the regression is estimating using the size as the only independent variable. Secondly, beta and book-to-market ratio sorting variables are added as well as the control independent variable. This regression is based on the Fama-MacBeth theory.  

#### (i) Effect of size independently

In [23]:
fama_macbeth_indep <- function(monthly_size=monthly_sizes, monthly_ret=monthly_returns) {
    # Initialize an xts object for storing regression results.
    fm_cs_results_indep <- as.xts(
        matrix(
            nrow = nrow(monthly_size) - 1,
            ncol = 5
        ),
        order.by = index(monthly_size[2:nrow(monthly_size)])
    )
    names(fm_cs_results_indep) <- c('Intercept', 'Size', 'R^2', 'Adjusted R^2', 'N')

    # Iterate over rows and perform cross-sectional regressions.
    for (i in 1:nrow(fm_cs_results_indep)) { # For each month
        current_month <- index(fm_cs_results_indep[i])
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

        if (!next_month %in% index(monthly_ret)) {
            next
        }

        # Avoid look-ahead bias by selecting the returns from the next month.
        month_returns <- monthly_ret[next_month]
        month_sizes <- monthly_size[current_month]

        # Save regression coefficients and other statistics.
        model <- lm(t(month_returns) ~ 1 + t(month_sizes))
        fm_cs_results_indep[i, 1] <- model$coefficients[[1]]
        fm_cs_results_indep[i, 2] <- model$coefficients[[2]]
        fm_cs_results_indep[i, 3] <- summary(model)$r.squared
        fm_cs_results_indep[i, 4] <- summary(model)$adj.r.squared
        fm_cs_results_indep[i, 5] <- nobs(model)
    }

    # Compute time series means, standard errors.
    fm_results_indep <- as.data.frame(matrix(nrow = 2, ncol = 2))
    names(fm_results_indep) <- c('Intercept', 'Size')

    for (i in 1:ncol(fm_results_indep)) {
        model <- lm(na.omit(fm_cs_results_indep[, i]) ~ 1)
        fm_results_indep[1, i] <- model$coefficients[[1]]
        fm_results_indep[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6)[[1]])
    }

    return (list(cs_results=fm_cs_results_indep, ts_results=fm_results_indep))
}

In [24]:
# Independent Size effect
results_indep <- fama_macbeth_indep()

fm_cs_indep <- results_indep$cs_results 
fm_indep <- results_indep$ts_results

#### (ii) Effect of size with Beta and Book-to-Market ratios controls

In [25]:
fama_macbeth_dep <- function(monthly_ret=monthly_returns, monthly_size=monthly_sizes, monthly_book=monthly_book_to_market, monthly_beta=monthly_betas) {
    # Correct indexing
    index(monthly_book) <- index(monthly_size)
    index(monthly_beta) <- index(monthly_size)
    # Remove missing values
    monthly_ret <- monthly_ret[(13:nrow(monthly_ret)),]
    monthly_size <- monthly_size[(13:nrow(monthly_size)),]
    monthly_book <- monthly_book[(13:nrow(monthly_book)),]
    monthly_beta <- monthly_beta[(13:nrow(monthly_beta)),]
    
    # Initialize an xts object for storing regression results.
    fm_cs_results_dep <- as.xts(
        matrix(
            nrow = nrow(monthly_size) - 1,
            ncol = 7
        ),
        order.by = index(monthly_size[2:nrow(monthly_size)])
    )
    names(fm_cs_results_dep) <- c('Intercept', 'Size', 'Book_to_market', 'Beta', 'R^2', 'Adjusted R^2', 'N')

    # Iterate over rows and perform cross-sectional regressions.
    for (i in 1:nrow(fm_cs_results_dep)) { # For each month
        current_month <- index(fm_cs_results_dep[i])
        next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

        if (!next_month %in% index(monthly_ret)) {
            next
        }

        # Avoid look-ahead bias by selecting the returns from the next month.
        month_returns <- monthly_ret[next_month]
        month_sizes <- monthly_size[current_month]
        month_book_to_market <- monthly_book[current_month]
        month_betas <- monthly_beta[i]
        
        # Save regression coefficients and other statistics.
        model <- lm(t(month_returns) ~ 1 + t(month_sizes) + t(month_book_to_market) + t(month_betas))
        fm_cs_results_dep[i, 1] <- model$coefficients[[1]]
        fm_cs_results_dep[i, 2] <- model$coefficients[[2]]
        fm_cs_results_dep[i, 3] <- model$coefficients[[3]]
        fm_cs_results_dep[i, 4] <- model$coefficients[[4]]
        fm_cs_results_dep[i, 5] <- summary(model)$r.squared
        fm_cs_results_dep[i, 6] <- summary(model)$adj.r.squared
        fm_cs_results_dep[i, 7] <- nobs(model)
    }

    # Compute time series means, standard errors.
    fm_results_dep <- as.data.frame(matrix(nrow = 2, ncol = 4))
    names(fm_results_dep) <- c('Intercept', 'Size', 'Book_to_market', 'Beta')

    for (i in 1:ncol(fm_results_dep)) {
        model <- lm(na.omit(fm_cs_results_dep[, i]) ~ 1)
        fm_results_dep[1, i] <- model$coefficients[[1]]
        fm_results_dep[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6)[[1]])
    }

    return (list(cs_results=fm_cs_results_dep, ts_results=fm_results_dep))
}

In [26]:
# Dependent Size effect
results_dep <- fama_macbeth_dep()

fm_cs_dep <- results_dep$cs_results 
fm_dep <- results_dep$ts_results

## 4. Results

### a) Average values

#### (i) Beta

In [27]:
print("Equally-weighted portfolio")
print("EW: Betas")
EW_portfolio_results_betas

print("Value-weighted portfolio")
print("VW: Betas")
VW_portfolio_results_betas

[1] "Equally-weighted portfolio"
[1] "EW: Betas"


,1,2,3,4,5,5 - 1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
average return,-0.06377180,-0.06052285,-0.06141061,-0.05899512,-0.05699457,0.006777229
average return t-stat,-1.51869041,-1.67469814,-1.96432656,-1.99633931,-2.13892704,1.436404507
CAPM alpha,-0.06410271,-0.06053159,-0.06122202,-0.05861997,-0.05638835,0.007714359
CAPM alpha t-stat,-1.40999664,-1.61085834,-1.85645638,-1.88384563,-2.00929480,1.450344729
FF alpha,-0.06255644,-0.05888060,-0.05968839,-0.05698713,-0.05511059,0.007445851
FF alpha t-stat,-1.91372269,-2.01891553,-2.18697776,-2.15732151,-2.20902128,1.355443148


[1] "Value-weighted portfolio"
[1] "VW: Betas"


,1,2,3,4,5,5 - 1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
average return,-0.06204732,-0.05988855,-0.06145291,-0.06004279,-0.05626927,0.005778046
average return t-stat,-1.38372566,-1.65190810,-1.94482778,-2.03209850,-2.05673076,1.132946243
CAPM alpha,-0.06207436,-0.05971612,-0.06183813,-0.05933391,-0.05592101,0.006153351
CAPM alpha t-stat,-1.31258105,-1.60352415,-1.91683738,-1.90231745,-1.95522007,1.107815996
FF alpha,-0.06025231,-0.05796250,-0.06019648,-0.05799401,-0.05284523,0.007407077
FF alpha t-stat,-1.78263983,-1.97336803,-2.26951365,-2.23615044,-2.08741986,1.281027643


Beta coefficient captures the riskiness of a stock. In orther words, beta measures how an individual stock reacts if the overall market, e.g., increases by 1%. We can observe a sligh increasing trend of an average coefficient (getting a little bit less negative) for higher portfolios, which would confirm that stocks with higher beta coefficients (riskier) should have also higher average returns. However, most of our t-statistic do not overstep the +-2.0 level and thus do not seem to be statistically significant. 

Concenring the 5-1 portfolio with the t-statistic being not significant, we cannot certainly find a long and short positions on the portfolios. Although the alpha coefficient are very similar across all methods and weighting scheme, the FF model controlling for several market factors reveals the largest t-statistics that are almost behind the significance level - This suggests that our results could be significant once correclty correcting for all market factors. We can also see that coefficient estimated by the equally-weighted scheme tend to be "more negative" signalizing that larger firms affecting the total average less in this scheme could have higher returns (they are underestimated in the equal-weighted scheme). 

In conclusion, we cannot directly confirm the riskiness-return relationship known from theory, however, we probably could controlling better for market factors.  

#### (ii) Size

In [28]:
print("Equally-weighted portfolio")
print("EW: Size")
EW_portfolio_results_size

print("Value-weighted portfolio")
print("VW: Size")
VW_portfolio_results_size

[1] "Equally-weighted portfolio"
[1] "EW: Size"


,1,2,3,4,5,5 - 1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
average return,-0.04970802,-0.05766072,-0.05837361,-0.06049427,-0.06068055,-0.01097253
average return t-stat,-1.75778952,-1.89563405,-1.92469916,-2.03035576,-1.96403606,-6.16006663
CAPM alpha,-0.04885616,-0.05748662,-0.05790278,-0.05999811,-0.06040633,-0.01155016
CAPM alpha t-stat,-1.57962996,-1.78437972,-1.75846761,-1.86482382,-1.84557181,-5.49064573
FF alpha,-0.04772586,-0.05594056,-0.05616774,-0.05859045,-0.05851768,-0.01079182
FF alpha t-stat,-1.79737518,-2.06508515,-2.05329291,-2.17393914,-2.15727982,-4.58075234


[1] "Value-weighted portfolio"
[1] "VW: Size"


,1,2,3,4,5,5 - 1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
average return,-0.04886994,-0.05523114,-0.05594754,-0.05861174,-0.05730864,-0.008438698
average return t-stat,-1.69252316,-1.81296433,-1.83831627,-1.94488497,-1.86299477,-3.537890743
CAPM alpha,-0.04803289,-0.05497732,-0.05537032,-0.05805747,-0.05690813,-0.008875243
CAPM alpha t-stat,-1.53673395,-1.70539304,-1.67766527,-1.78707057,-1.76532060,-3.362262369
FF alpha,-0.04660500,-0.05338942,-0.05358991,-0.05658790,-0.05484845,-0.008243454
FF alpha t-stat,-1.75416765,-1.97284107,-1.96478883,-2.09966167,-2.03101372,-2.918352958


We can see the decreasing average alpha coefficient for the size-sorted portfolios. In other words, larger firms concentrated in portfolio 4 and 5 tend to have lower average returns than the smaller firms in portfolios 1 and 2. This observation supports a theoretical assumption of smaller companies being riskier (e.g. due to their higher probability of default and lower stability in market environment) and thus having higher returns to compensate investor for this riskiness. Same as before, the t-statistic are not significant for the simple average and CAPM model alpha and only slightly significant for the FF model. 

Since value-weighted portfolio seems to have a little bit "less negative" average returns once again, there could be a small skewness due to underestimating of positive returns of larger companies. Since t-statistic on 5-1 portfolio is significant in this case, we can conclude that an investor should have a long position on portfolio 1 and short position on 5.  

#### (iii) Book-to-market ratio

In [29]:
print("Equally-weighted portfolio")
print("EW: Book to Market")
EW_portfolio_results_book_to_market

print("Value-weighted portfolio")
print("VW: Book to Market")
VW_portfolio_results_book_to_market

[1] "Equally-weighted portfolio"
[1] "EW: Book to Market"


,1,2,3,4,5,5 - 1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
average return,-0.05364340,-0.05803746,-0.05777019,-0.05998162,-0.05776707,-0.004123672
average return t-stat,-1.81429372,-1.95222884,-1.92196083,-1.91019918,-2.07196804,-1.477755728
CAPM alpha,-0.05331438,-0.05780123,-0.05729727,-0.05955687,-0.05701852,-0.003704138
CAPM alpha t-stat,-1.70535600,-1.86371009,-1.78321959,-1.71326651,-1.87533594,-1.254041413
FF alpha,-0.05193733,-0.05612682,-0.05566016,-0.05801988,-0.05555621,-0.003618878
FF alpha t-stat,-1.90698647,-2.08074219,-2.06078410,-2.10233648,-2.17113318,-1.188614054


[1] "Value-weighted portfolio"
[1] "VW: Book to Market"


,1,2,3,4,5,5 - 1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
average return,-0.05328468,-0.05799912,-0.05840982,-0.06041282,-0.06007420,-0.006789522
average return t-stat,-1.78108284,-1.85719511,-1.77304584,-1.92877728,-2.22017318,-1.687851494
CAPM alpha,-0.05315330,-0.05756443,-0.05794356,-0.05959824,-0.05925900,-0.006105701
CAPM alpha t-stat,-1.72921462,-1.76414661,-1.63667789,-1.73480509,-2.04983754,-1.391294236
FF alpha,-0.05121654,-0.05579415,-0.05596943,-0.05772971,-0.05775947,-0.006542932
FF alpha t-stat,-1.92208352,-2.02274944,-1.96914090,-2.10333189,-2.32058667,-1.527890430


With the average returns coefficients having slightly decreasing size for companies with higher book-to-market ratio. This could suggest that the "growth stocks" have a bit higher returns than the "value stocks" that goes hand by hand with a theoretical assumption. The t-statistics are once again significant only in some cases, mainly for the FF-model controlling for market factors. Since neither of the t-statistic on the 5-1 portfolio is significant, we cannot say anything about the long and short position. The coefficients are once again "less negative" using the value-weighted scheme, however, this difference is very small in this case.   

### b) Fama-MacBeth regression

#### (i) Effect of size independently

In [30]:
print("Independent size effect")
head(fm_cs_indep)
head(fm_indep)

[1] "Independent size effect"


             Intercept         Size          R^2 Adjusted R^2   N
2010-02-28  0.08537439 -0.005167857 0.0103693027  0.006121961 235
2010-03-31  0.07199080 -0.007870397 0.0200350250  0.015829167 235
2010-04-30 -0.02202517 -0.010164942 0.0392331228  0.035091886 234
2010-05-31 -0.04748911 -0.001683906 0.0011192495 -0.003186271 234
2010-06-30  0.04969456  0.001772287 0.0009148128 -0.003373106 235
2010-07-31 -0.04037301 -0.002215755 0.0015456333 -0.002685106 238

,Intercept,Size
,<dbl>,<dbl>
1,-0.03829303,-0.00323626
2,-1.41018457,-6.15458626


The Fama-MacBeth regression results found a statistically significant negative affect of size on portfolio average returns. Therefore, we can conclude that larger firms have on average slightly lower one-month-advance returns that these with lower size. This finding corresponds also to the previous finding above with the size being the sorting variable. Theoretically, investing in smaller firms should be rewarded by higher returns since this investment is riskier in terms of, for example, higher probability of company's default, higher fluctuation due to the market conditions. In the iinvesting point of view, size as a relevant factor for company's future returns could be theoretically used for predictions and investment strategy. 

#### (ii) Effect of size with Beta and Book-to-Market ratios controls

In [31]:
print("Dependent size effect")
head(fm_cs_dep)
head(fm_dep)

[1] "Dependent size effect"


             Intercept         Size Book_to_market       Beta        R^2
2011-02-28  0.11113845 -0.015989660   -0.017170245  -0.706583 0.13729002
2011-03-31  0.08177565 -0.003668668   -0.009477152  -1.932728 0.03355343
2011-04-30  0.08731898 -0.005650246   -0.014586434  -4.863351 0.12386782
2011-05-31  0.02351362 -0.001817961   -0.012888975  -1.666071 0.04050094
2011-06-30 -0.03098609  0.003023331    0.001967405  -2.115652 0.02077328
2011-07-31  0.13579248 -0.010157504   -0.012560894 -13.098052 0.33812838
           Adjusted R^2   N
2011-02-28  0.126134283 236
2011-03-31  0.021002181 235
2011-04-30  0.112538523 236
2011-05-31  0.028146878 237
2011-06-30  0.008219094 238
2011-07-31  0.329678959 239

,Intercept,Size,Book_to_market,Beta
,<dbl>,<dbl>,<dbl>,<dbl>
1,-0.05192624,-0.002689226,-0.004633392,0.6522987
2,-1.69106467,-6.350258873,-2.016458064,1.3329654


Incorporating book-to-market ratio and riskiness (beta) to the Fama-Macbeth regression as control variables did not change the coefficient significance. Since all the sorting variable have a significant effect on future returns going in an expected direction, we can once again conclude that the larger companies tend to have smaller average returns. The effect being slightly 'less negative' that in the previous case shows that this could be overestimated without controlling for other market factors.

## 5. Conclusion

In this analysis, 250 stocks were divided into 5 portfolios based on three sorting variables (beta coefficient, size, book-to-market ratio) and their average returns calculated using the equally and value-weighted methods. The average returns measured by all sorting variables as well as Fama-MacBeth regression were utilized to estimate an effect of sorting variables on company's future stock returns.

We found a statistically significant effect of size on company's future returns as predicted by standard theory suggesting that smaller companies reward investors by higher average returns mainly due to their higher riskiness. On the other hand, calculating the average values by employing standard average, CAPM model, and FF model did not reveal or only slighlty significan estimates. In conclusion, we attemted to confirm the theoretical assumption of size and book-to-market ratio going opposite the direction of average returns as well as riskiness (beta) going in the same direction. 